#  CAMEL Cookbook: SQL MCP Server

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1fZHCfDqEEseewSZ2TQSJiAA3il9VT4ut?usp=sharing) 

<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)
</div>

### Note on MCP Server Implementation

There are *two approaches* to implementing an **MCP server**:

- **TypeScript with JSON Config:**
This approach is typically used with npm-based clients (such as Cursor or Claude Desktop), where a config.json file is used to configure and connect to the server.

- **Python SDK Approach:**
    In this notebook, we use the Python SDK. Here, you write the MCP server in Python and connect directly without a config.json file.

This notebook demonstrates how to set up and leverage CAMEL's MCP toolkit to create a natural language interface for SQL database operations.

In this notebook, you'll explore:
* **CAMEL**: A multi-agent framework enabling sophisticated AI-driven tasks
* **MCP Server**: Custom tools for SQL database operations (querying, listing tables, schema exploration)
* **SQLite Integration**: Practical database implementation with sample data
* **Claude LLM**: Natural language understanding and query translation

This setup serves as a flexible framework adaptable for various scenarios requiring database access through natural language.

## 📦 Installation

First, install the CAMEL package with all its dependencies:

In [ ]:
%pip install "camel-ai[all]==0.2.31"

## 🔑 Setting Up API Keys

You'll need to set up your API keys for **Anthropic**

In [ ]:
import os
from getpass import getpass

# Prompt for the API key securely
anthropic_api_key = getpass('Enter your API key: ')
os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key

Enter your API key: ··········


## Understanding MCP in CAMEL

MCP (Model Control Protocol) is a key feature in CAMEL,`MCPToolkit` class allows language models to interact with external tools and services. In this cookbook, we'll create an MCP server that provides SQL database capabilities to Claude through CAMEL.

Our MCP server will support the following operations:
- Creating SQLite databases
- Listing tables in a database
- Describing table schemas
- Executing SQL queries (SELECT, INSERT, UPDATE, DELETE)

Let's start by creating our SQL MCP server

## Building the SQL MCP Server

Now we'll build our SQL MCP server. This server will expose tools for database operations that our CAMEL agent can use. Let's create our server file:

In [ ]:
# SQL MCP Server for CAMEL

import os
import asyncio  # noqa: F401
import sqlite3
import json
from mcp.server.fastmcp import FastMCP # type: ignore
from camel.logger import get_logger # type: ignore

logger = get_logger(__name__) # initiating logger
mcp = FastMCP("sqldb") # create a new instance of FastMCP with name sqldb

The code above sets up the basic structure for our MCP server. We:

1. Import necessary libraries
2. Set up logging
3. Initialize the MCP server with the name "sqldb"

### Adding Database Query Tool
Now let's add our first tool - the ability to execute SQL queries:

In [3]:
# Add to sql_server_mcp.py
@mcp.tool()
async def execute_query(connection_string: str, query: str) -> str:
    r"""Executes the SQL query on the given database.
    Args:
        connection_string (str): The connection string or path to the SQLite database.
        query (str): The SQL query to execute.
    Returns:
        str: The result of the query as a JSON string, or an error message if execution fails.
    """
    logger.info(f"execute_query triggered with connection_string: {connection_string}")
    # For security reasons, don't log the full query in production
    logger.info(f"Query starts with: {query[:20]}...")
    
    try:
        # For this example, we'll use SQLite which just takes a file path
        conn = sqlite3.connect(connection_string)
        cursor = conn.cursor()
        
        # Execute the query
        cursor.execute(query)
        
        # Check if this is a SELECT query (has results to fetch)
        if query.strip().upper().startswith("SELECT"):
            # Get column names from cursor description
            columns = [desc[0] for desc in cursor.description]
            
            # Fetch results and format as a list of dictionaries
            results = []
            for row in cursor.fetchall():
                results.append(dict(zip(columns, row)))
            
            conn.close()
            return json.dumps(results, indent=2)
        else:
            # For INSERT, UPDATE, DELETE, etc.
            conn.commit()
            affected_rows = cursor.rowcount
            conn.close()
            return json.dumps({"affected_rows": affected_rows}, indent=2)
    
    except Exception as e:
        return f"Error executing SQL query: {e}"

execute_query.inputSchema = {
    "type": "object",
    "properties": {
         "connection_string": {
             "type": "string",
             "title": "Connection String",
             "description": "The connection string or path to the SQLite database."
         },
         "query": {
             "type": "string",
             "title": "SQL Query",
             "description": "The SQL query to execute."
         }
    },
    "required": ["connection_string", "query"]
}


This tool handles:

- Connecting to a SQLite database
- Executing a SQL query
- Handling different types of queries (SELECT vs. non-SELECT)
- Formatting results as JSON
- Error handling

The `inputSchema` defines the required parameters and provides descriptions that help the LLM understand how to use the tool.

### Adding List Tables Tool

Next, let's add a tool to list all tables in a database:

In [4]:
# Add to sql_server_mcp.py
@mcp.tool()
async def list_tables(connection_string: str) -> str:
    r"""Lists all tables in the specified database.
    Args:
        connection_string (str): The connection string or path to the SQLite database.
    Returns:
        str: A JSON string containing the list of tables, or an error message if listing fails.
    """
    logger.info(f"list_tables triggered with connection_string: {connection_string}")
    
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(connection_string)
        cursor = conn.cursor()
        
        # Query to get all table names in SQLite
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        
        # Fetch and format results
        tables = [row[0] for row in cursor.fetchall()]
        conn.close()
        
        return json.dumps({"tables": tables}, indent=2)
    
    except Exception as e:
        return f"Error listing tables: {e}"

list_tables.inputSchema = {
    "type": "object",
    "properties": {
         "connection_string": {
             "type": "string",
             "title": "Connection String",
             "description": "The connection string or path to the SQLite database."
         }
    },
    "required": ["connection_string"]
}

This tool allows the LLM to discover what tables exist in a database, which is essential for database exploration.

### Adding Database Creation Tool

Now we'll add a tool to create new SQLite databases:

In [5]:
# Add to sql_server_mcp.py
@mcp.tool()
async def create_database(db_path: str) -> str:
    r"""Creates a new SQLite database at the specified path.
    Args:
        db_path (str): The path where the new database should be created.
    Returns:
        str: A success message or an error message if creation fails.
    """
    logger.info(f"create_database triggered with db_path: {db_path}")
    
    try:
        # Check if file already exists
        if os.path.exists(db_path):
            return f"Database already exists at {db_path}"
        
        # Create a new SQLite database by connecting to it
        conn = sqlite3.connect(db_path)
        conn.close()
        
        return json.dumps({"status": "success", "message": f"Database created at {db_path}"}, indent=2)
    
    except Exception as e:
        return f"Error creating database: {e}"

create_database.inputSchema = {
    "type": "object",
    "properties": {
         "db_path": {
             "type": "string",
             "title": "Database Path",
             "description": "The path where the new SQLite database should be created."
         }
    },
    "required": ["db_path"]
}

This simple tool creates empty SQLite databases that can later be populated with tables and data.

### Adding Table Description Tool

Finally, let's add a tool to describe table schemas:

In [6]:
# Add to sql_server_mcp.py
@mcp.tool()
async def describe_table(connection_string: str, table_name: str) -> str:
    r"""Describes the schema of a specified table.
    Args:
        connection_string (str): The connection string or path to the SQLite database.
        table_name (str): The name of the table to describe.
    Returns:
        str: A JSON string containing the table schema, or an error message if the operation fails.
    """
    logger.info(f"describe_table triggered with connection_string: {connection_string}, table_name: {table_name}")
    
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(connection_string)
        cursor = conn.cursor()
        
        # Query to get table schema in SQLite
        cursor.execute(f"PRAGMA table_info({table_name});")
        
        # Fetch and format results
        columns = []
        for row in cursor.fetchall():
            columns.append({
                "cid": row[0],
                "name": row[1],
                "type": row[2],
                "notnull": row[3],
                "default_value": row[4],
                "pk": row[5]
            })
        
        conn.close()
        
        return json.dumps({"table": table_name, "columns": columns}, indent=2)
    
    except Exception as e:
        return f"Error describing table: {e}"

describe_table.inputSchema = {
    "type": "object",
    "properties": {
         "connection_string": {
             "type": "string",
             "title": "Connection String",
             "description": "The connection string or path to the SQLite database."
         },
         "table_name": {
             "type": "string",
             "title": "Table Name",
             "description": "The name of the table to describe."
         }
    },
    "required": ["connection_string", "table_name"]
}

This tool retrieves column details for a specified table, which helps the LLM understand the database structure before running queries.

### Adding Main Function

To complete our MCP server, let's add the main function:

In [ ]:
# Add to sql_server_mcp.py
def main(transport: str = "stdio"):
    r"""Runs the SQL MCP Server.
    This server provides SQL database functionalities via MCP.
    Args:
        transport (str): The transport mode ('stdio' or 'sse').
    """
    if transport == 'stdio':
        mcp.run(transport='stdio')
    elif transport == 'sse':
        mcp.run(transport='sse')
    else:
        print(f"Unknown transport mode: {transport}")

if __name__ == "__main__":
    import sys
    transport_mode = sys.argv[1] if len(sys.argv) > 1 else "stdio"
    main(transport_mode)

Save this script as `sql_server_mcp.py` and run it using the command:
```bash
python sql_server_mcp.py
```

## Creating a Sample Database

Now that we have our MCP server, let's create a sample database to demonstrate its capabilities:

In [ ]:
import os

# Function to create a sample database
def create_sample_database(db_path="sample.db"):
    """Create a sample SQLite database with employees and departments tables."""
    
    # If database already exists, remove it to start fresh
    if os.path.exists(db_path):
        os.remove(db_path)
    
    print(f"Creating sample database at: {db_path}")
    
    # Create the database and add sample tables and data
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create employees table
    cursor.execute("""
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department TEXT,
        salary REAL,
        hire_date TEXT
    )
    """)
    
    # Insert sample employee data
    employees = [
        (1, 'John Doe', 'Engineering', 85000.00, '2020-01-15'),
        (2, 'Jane Smith', 'Marketing', 75000.00, '2019-05-20'),
        (3, 'Bob Johnson', 'Engineering', 95000.00, '2018-11-10'),
        (4, 'Alice Brown', 'HR', 65000.00, '2021-03-05'),
        (5, 'Charlie Davis', 'Engineering', 90000.00, '2020-08-12')
    ]
    cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?, ?)", employees)
    
    print("Added 5 employees to the database")
    
    # Create departments table
    cursor.execute("""
    CREATE TABLE departments (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        budget REAL,
        location TEXT
    )
    """)
    
    # Insert sample department data
    departments = [
        (1, 'Engineering', 1000000.00, 'Building A'),
        (2, 'Marketing', 500000.00, 'Building B'),
        (3, 'HR', 300000.00, 'Building A'),
        (4, 'Finance', 600000.00, 'Building C')
    ]
    cursor.executemany("INSERT INTO departments VALUES (?, ?, ?, ?)", departments)
    
    print("Added 4 departments to the database")
    
    # Commit changes and close connection
    conn.commit()
    conn.close()
    
    print(f"Sample database created successfully at: {db_path}")
    return db_path

# Create our sample database
db_path = create_sample_database()

Creating sample database at: sample.db
Added 5 employees to the database
Added 4 departments to the database
Sample database created successfully at: sample.db


## Connecting CAMEL to Our SQL MCP Server

Now let's set up a function to create a CAMEL agent that uses our SQL MCP server:

In [ ]:
import sys
from pathlib import Path

from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.toolkits import MCPToolkit
from camel.types import ModelPlatformType
from camel.toolkits.mcp_toolkit import _MCPServer

async def setup_sql_agent(db_path):
    """Set up a CAMEL agent with SQL MCP tools"""
    
    # Determine the path to the server file
    server_script_path = Path.cwd() / "sql_server_mcp.py"
    if not server_script_path.is_file():
        print(f"Error: Server script not found at {server_script_path}")
        return None
    
    print(f"Using MCP server script at: {server_script_path}")
        
    # Create an _MCPServer instance for our SQL server
    server = _MCPServer(
        command_or_url=sys.executable,
        args=[str(server_script_path)]
    )
    
    print("Created MCP server instance")
    
    # Create the MCP toolkit with our server
    mcp_toolkit = MCPToolkit(servers=[server])
    
    print("Created MCP toolkit")
    
    # Connect to the toolkit and get the tools
    async with mcp_toolkit.connection() as toolkit:
        tools = toolkit.get_tools()
        
        print("Connected to MCP toolkit and retrieved tools")
        
        # Create the system message for Claude
        sys_msg = (
            "You are a helpful SQL assistant. Use the provided external tools for database operations. "
            "Always use the tools to query the database rather than answering from your general knowledge. "
            f"The sample database is at '{db_path}'. It contains tables for employees and departments. "
            "When a user asks about the database, always use list_tables first to understand the schema, "
            "then use describe_table to see column details before executing queries. "
            "Always show your queries and explain your approach."
        )
        
        # Set up the model (Claude)
        model = ModelFactory.create(
            model_platform=ModelPlatformType.ANTHROPIC,
            model_type="claude-3-7-sonnet-20250219",
            api_key=os.getenv("ANTHROPIC_API_KEY"),
            model_config_dict={"temperature": 0.5, "max_tokens": 4096},
        )
        
        print("Created Claude model instance")
        
        # Create the CAMEL agent
        camel_agent = ChatAgent(
            system_message=sys_msg,
            model=model,
            tools=tools,
        )
        
        print("Created CAMEL agent with SQL tools")
        
        return camel_agent, toolkit

# This doesn't run the agent yet, just defines the setup function
print("Agent setup function defined. Ready for examples!")

Agent setup function defined. Ready for examples!


## Creating a Complete Example Runner

For convenience, let's create a script that sets up a complete interactive SQL MCP example:

In [15]:
%%writefile sql_example_run.py
import asyncio
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.toolkits import MCPToolkit
from camel.types import ModelPlatformType
from camel.toolkits.mcp_toolkit import _MCPServer

# Load environment variables from .env file
load_dotenv()

# Set your Anthropic API key (ensure this is valid in your .env file)
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

# Create a sample database for demonstration
async def create_sample_database():
    """Create a sample SQLite database with some data for demonstration."""
    import sqlite3
    
    # Create a temporary database in the current directory
    db_path = "sample.db"
    
    # If database already exists, remove it to start fresh
    if os.path.exists(db_path):
        os.remove(db_path)
    
    # Create the database and add sample tables and data
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create employees table
    cursor.execute("""
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department TEXT,
        salary REAL,
        hire_date TEXT
    )
    """)
    
    # Insert sample employee data
    employees = [
        (1, 'John Doe', 'Engineering', 85000.00, '2020-01-15'),
        (2, 'Jane Smith', 'Marketing', 75000.00, '2019-05-20'),
        (3, 'Bob Johnson', 'Engineering', 95000.00, '2018-11-10'),
        (4, 'Alice Brown', 'HR', 65000.00, '2021-03-05'),
        (5, 'Charlie Davis', 'Engineering', 90000.00, '2020-08-12')
    ]
    cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?, ?)", employees)
    
    # Create departments table
    cursor.execute("""
    CREATE TABLE departments (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        budget REAL,
        location TEXT
    )
    """)
    
    # Insert sample department data
    departments = [
        (1, 'Engineering', 1000000.00, 'Building A'),
        (2, 'Marketing', 500000.00, 'Building B'),
        (3, 'HR', 300000.00, 'Building A'),
        (4, 'Finance', 600000.00, 'Building C')
    ]
    cursor.executemany("INSERT INTO departments VALUES (?, ?, ?, ?)", departments)
    
    # Commit changes and close connection
    conn.commit()
    conn.close()
    
    print(f"Sample database created at: {db_path}")
    return db_path

# Interactive mode function to chat with the agent
async def interactive_input_loop(agent: ChatAgent, db_path: str):
    loop = asyncio.get_event_loop()
    print("\n==== SQL Assistant Interactive Mode ====")
    print("Type 'exit' at any prompt to quit.")
    print(f"\nUsing sample database at: {db_path}")
    print("\nSample queries you can try:")
    print("- Show me all tables in sample.db")
    print("- What columns are in the employees table in sample.db?")
    print("- List all employees in the Engineering department")
    print("- What is the average salary by department?")
    print("- How many employees are in each department?")
    print("- Find the employee with the highest salary")
    print("- Add a new employee named Michael Wilson to Finance with salary 82000")
    print("======================================")

    while True:
        query = await loop.run_in_executor(
            None, 
            input, 
            "\nEnter your query (or type 'exit' to quit): "
        )
        
        if query.lower() == 'exit':
            print("Exiting interactive mode.")
            break
        
        print("\nProcessing query...")
        response = await agent.astep(query)
        
        print("\nAgent Response:")
        if response.msgs and response.msgs[0].content:
            print(response.msgs[0].content.rstrip())
        else:
            print("No output received.")

# Main function to run the entire example
async def main(server_transport: str = 'stdio'):
    # First create a sample database
    db_path = await create_sample_database()
    
    if server_transport == 'stdio':
        # Determine the path to the server file
        server_script_path = Path(__file__).resolve().parent / "sql_server_mcp.py"
        if not server_script_path.is_file():
            print(f"Error: Server script not found at {server_script_path}")
            return
            
        # Create an _MCPServer instance for our SQL server
        server = _MCPServer(
            command_or_url=sys.executable,
            args=[str(server_script_path)]
        )
        mcp_toolkit = MCPToolkit(servers=[server])
    else:
        mcp_toolkit = MCPToolkit("tcp://localhost:5000")

    async with mcp_toolkit.connection() as toolkit:
        tools = toolkit.get_tools()
        sys_msg = (
            "You are a helpful SQL assistant. Use the provided external tools for database operations. "
            "Always use the tools to query the database rather than answering from your general knowledge. "
            f"The sample database is at '{db_path}'. It contains tables for employees and departments. "
            "When a user asks a question about the database, ALWAYS explicitly include the database path "
            f"'{db_path}' in your tool calls. First list the tables to understand the schema, "
            "then use describe_table to see column details before querying."
        )
        model = ModelFactory.create(
            model_platform=ModelPlatformType.ANTHROPIC,
            model_type="claude-3-7-sonnet-20250219",
            api_key=os.getenv("ANTHROPIC_API_KEY"),
            model_config_dict={"temperature": 0.5, "max_tokens": 4096},
        )
        camel_agent = ChatAgent(
            system_message=sys_msg,
            model=model,
            tools=tools,
        )
        camel_agent.reset()
        camel_agent.memory.clear()
        await interactive_input_loop(camel_agent, db_path)
        
        # Clean up the sample database after we're done
        if os.path.exists(db_path):
            os.remove(db_path)
            print(f"\nRemoved sample database: {db_path}")

# Entry point 
if __name__ == "__main__":
    asyncio.run(main())

Overwriting sql_example_run.py


# Output - 

Entering interactive mode. Type 'exit' at any prompt to quit.

Using sample database at: sample.db

You can try commands like:
- Show me all tables in the database
- List all employees
- What is the average salary by department?
- How many employees are in the Engineering department?
- Find the employee with the highest salary

Enter your query (or type 'exit' to quit): Show me all the tables in the database

Agent Response:
I'll help you list all the tables in the database. However, I need to know which database you want to examine. Could you please provide the connection string or path to the SQLite database?

Enter your query (or type 'exit' to quit): list all employees in sample.db

Agent Response:
Here are all the employees from the sample.db database:

| ID | Name | Department | Salary | Hire Date |
|----|------|------------|--------|-----------|
| 1 | John Doe | Engineering | $85,000 | 2020-01-15 |
| 2 | Jane Smith | Marketing | $75,000 | 2019-05-20 |
| 3 | Bob Johnson | Engineering | $95,000 | 2018-11-10 |
| 4 | Alice Brown | HR | $65,000 | 2021-03-05 |
| 5 | Charlie Davis | Engineering | $90,000 | 2020-08-12 |

The database contains 5 employees across 3 departments (Engineering, Marketing, and HR).

Enter your query (or type 'exit' to quit): find the employee with the lowest salary 

Agent Response:
The employee with the lowest salary is:

| ID | Name | Department | Salary | Hire Date |
|----|------|------------|--------|-----------|
| 4 | Alice Brown | HR | $65,000 | 2021-03-05 |

Alice Brown from the HR department has the lowest salary at $65,000.

# Conclusion

To run it locally you can save your `sql_server_mcp.py` in the same directory of `sql_example_run.py` and then run the whole setup with :

`python sql_example_run.py`

This notebook has guided you through ***setting up and running a SQL MCP (Model Control Protocol) server with CAMEL AI*** to enable natural language interaction with SQL databases. 

The implementation allows *Claude* to understand and execute database operations through conversational prompts.

**Key tools** utilized in this notebook include:

- **CAMEL:** A powerful multi-agent framework that enables Retrieval-Augmented Generation and multi-agent role-playing scenarios, allowing for sophisticated AI-driven tasks.
- **MCP Server:** A protocol that enables language models to use external tools by defining standardized interfaces for tool execution.
- **SQLite:** A lightweight, file-based relational database used for demonstrating SQL operations in our examples.
- **Claude:** Anthropic's large language model that interprets natural language queries and translates them into appropriate database operations.
- **Python's asyncio:** Used for handling asynchronous operations between CAMEL, the MCP server, and database connections.

This comprehensive setup allows you to *adapt and expand the example for various scenarios involving database operations through natural language*. You could extend it to support other database systems, add more complex SQL operations, or integrate it with existing data workflows in your organization.

That's everything: Got questions about 🐫 CAMEL-AI? Join us on [Discord](https://discord.camel-ai.org)! Whether you want to share feedback, explore the latest in multi-agent systems, get support, or connect with others on exciting projects, we’d love to have you in the community! 🤝

Check out some of our other work:

1. 🐫 Creating Your First CAMEL Agent [free Colab](https://docs.camel-ai.org/cookbooks/create_your_first_agent.html)

2.  Graph RAG Cookbook [free Colab](https://colab.research.google.com/drive/1uZKQSuu0qW6ukkuSv9TukLB9bVaS1H0U?usp=sharing)

3. 🧑‍⚖️ Create A Hackathon Judge Committee with Workforce [free Colab](https://colab.research.google.com/drive/18ajYUMfwDx3WyrjHow3EvUMpKQDcrLtr?usp=sharing)

4. 🔥 3 ways to ingest data from websites with Firecrawl & CAMEL [free Colab](https://colab.research.google.com/drive/1lOmM3VmgR1hLwDKdeLGFve_75RFW0R9I?usp=sharing)

5. 🦥 Agentic SFT Data Generation with CAMEL and Mistral Models, Fine-Tuned with Unsloth [free Colab](https://colab.research.google.com/drive/1lYgArBw7ARVPSpdwgKLYnp_NEXiNDOd-?usp=sharingg)

Thanks from everyone at 🐫 CAMEL-AI


<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)
</div>
